#### CREDIT TO @JKGIESLER ON GITHUB.COM

In [1]:
import json
import os
import sklearn
import numpy
from sklearn.metrics import mean_squared_error
from collections import namedtuple
import sys
from math import log,tan,pi,radians
import pandas as pd
from sodapy import Socrata
from config import CHI_API_KEY
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import loadtxt
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
client = Socrata("data.cityofchicago.org", app_token = CHI_API_KEY)

results = client.get("85ca-t3if", limit= 10)
results_df = pd.DataFrame.from_records(results)

In [3]:
for col in results_df.columns: 
    print(col) 

crash_record_id
crash_date
posted_speed_limit
traffic_control_device
device_condition
weather_condition
lighting_condition
first_crash_type
trafficway_type
alignment
roadway_surface_cond
road_defect
report_type
crash_type
intersection_related_i
damage
date_police_notified
prim_contributory_cause
sec_contributory_cause
street_no
street_direction
street_name
beat_of_occurrence
num_units
most_severe_injury
injuries_total
injuries_fatal
injuries_incapacitating
injuries_non_incapacitating
injuries_reported_not_evident
injuries_no_indication
injuries_unknown
crash_hour
crash_day_of_week
crash_month
latitude
longitude
location
hit_and_run_i


In [4]:
Pt = namedtuple('Pt','x,y')
Edge = namedtuple('Edge','a,b')
Poly = namedtuple('Poly','name,edges')
_eps = 1e-5
_huge = sys.float_info.max
_tiny = sys.float_info.min

In [5]:
def load_json():
	file_in = open(os.getcwd()+"/Neighborhoods_2012_polygons.json")
	d = json.load(file_in)
	return d

In [6]:
def spherical_mercator_projection(longitude,latitude):
    x = -longitude
    y = log(tan(radians(pi/4 + latitude/2)))
    return (x,y)

In [7]:
def rayintersectseg(p, edge):
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = Pt(p.x, p.y + _eps)
    intersect = False
 
    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False
 
    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        
        intersect = m_blue >= m_red
    return intersect

In [8]:
def is_odd(x): 
	return x%2 == 1

def ispointinside(p, poly):
    ln = len(poly)
    return is_odd(sum(rayintersectseg(p, edge)
                    for edge in poly.edges ))

In [9]:
def get_all_neighborhoods():
	d = load_json()
	shape_list=[]
	for shape_idx in range(len(d['features'])):
		name = d['features'][shape_idx]['properties']['SEC_NEIGH']

		edges =[]
		for coordinate_idx in range(len(d['features'][shape_idx]['geometry']['coordinates'][0])-1):
			lon_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][0]
			lat_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][1]
			
			lon_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][0]
			lat_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][1]
			
			x1,y1 = spherical_mercator_projection(lon_1,lat_1)
			x2,y2 = spherical_mercator_projection(lon_2,lat_2)
			edges.append(Edge(a=Pt(x=x1,y=y1),b=Pt(x=x2,y=y2)))
		
		shape_list.append(Poly(name=name,edges=tuple(edges)))
	return shape_list

In [10]:
def find_neighborhood(test_long,test_lat,all_neighborhoods):
	x,y = spherical_mercator_projection(test_long,test_lat)
	for neighborhood in all_neighborhoods:
		correct_neighborhood = ispointinside(Pt(x=x,y=y),neighborhood)
		if correct_neighborhood:
			return neighborhood.name

all_neighborhoods = get_all_neighborhoods()

In [11]:
neighborhood = []
for row in results_df["longitude"]:
    test_long = float(row)
for row_1 in results_df["latitude"]:
    test_lat = float(row_1)
    neighborhood.append(find_neighborhood(test_long,test_lat,all_neighborhoods))

results_df['hood_label'] = neighborhood
results_df['hood_label']

0                             ENGLEWOOD
1                             LAKE VIEW
2                           ROGERS PARK
3           BRIGHTON PARK,MCKINLEY PARK
4                             ENGLEWOOD
5                        AUBURN GRESHAM
6                       LOWER WEST SIDE
7                             EDGEWATER
8                             LAKE VIEW
9    UKRAINIAN VILLAGE AND EAST VILLAGE
Name: hood_label, dtype: object

In [12]:
results_df['hood_label'].astype(str)


0                             ENGLEWOOD
1                             LAKE VIEW
2                           ROGERS PARK
3           BRIGHTON PARK,MCKINLEY PARK
4                             ENGLEWOOD
5                        AUBURN GRESHAM
6                       LOWER WEST SIDE
7                             EDGEWATER
8                             LAKE VIEW
9    UKRAINIAN VILLAGE AND EAST VILLAGE
Name: hood_label, dtype: object

In [13]:
ml_df = pd.DataFrame(results_df[["hood_label","posted_speed_limit", "crash_hour"]])
ml_df.dtypes

hood_label            object
posted_speed_limit    object
crash_hour            object
dtype: object

In [14]:

values = array(results_df["crash_hour"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
hour_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(hour_onehot_encoded)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [18]:
values = array(results_df["hood_label"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
damage_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

TypeError: argument must be a string or number

In [15]:
values = array(results_df["damage"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
damage_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [16]:
damage_onehot_encoded.dtype

dtype('float64')

In [17]:
# ml_df["posted_speed_limit"] = pd.to_numeric(ml_df["posted_speed_limit"])
# ml_df["crash_hour"] = pd.to_numeric(ml_df["crash_hour"])
# ml_df.dtypes

ml_df = pd.DataFrame([[damage_onehot_encoded, hour_onehot_encoded]],columns=['damage_onehot_encoded','hour_onehot_encoded'])

In [18]:
ml_df.to_dict('series')


{'damage_onehot_encoded': 0    [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
 Name: damage_onehot_encoded, dtype: object,
 'hour_onehot_encoded': 0    [[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...
 Name: hour_onehot_encoded, dtype: object}

In [24]:
### XGBoost Model



ml_df.to_dict('series')

X = ml_df["hour_onehot_encoded"]
y = ml_df["damage_onehot_encoded"]

data_dmatrix = xgb.DMatrix(data=X,label=y)

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

clf = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

clf.fit(X_train,y_train)

preds = clf.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

ValueError: setting an array element with a sequence.

In [25]:
### Logistic Regression Model 

X = ml_df["hour_onehot_encoded"]
y = ml_df["damage_onehot_encoded"]

clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

/Users/bdaves/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bdaves/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bdaves/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bdaves/opt/anaconda3/lib/python3.7/site-packages/tensorflow/

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [85]:

X = damage_onehot_encoded
y = hour_onehot_encoded
print(X.shape)
print(y.shape)

(10, 3)
(10, 4)


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [87]:
X_scaler = StandardScaler().fit(X_train)

In [88]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [89]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

array([[[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]]], dtype=float32)

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [91]:
model = Sequential()

In [92]:
number_inputs = 3
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [93]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [94]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 4)                 16        
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 10        
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=1000,
    shuffle=True,
    verbose=2
)

ValueError: A target array with shape (7, 4) was passed for an output of shape (None, 2) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [13]:
# lat = []
# lon = []

# for row in results_df['geo']:
#     try:
#         lat.append(row.split(',')[0])
#         lon.append(row.split(',')[1])
#     except:
#         lat.append(np.NaN)
#         lon.append(np.NaN)

# results_df['latitude'] = lat
# results_df['longitude'] = lon

In [25]:
# neighborhood = []

# for row in results_df:
#     try:
#         a = find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods)
#         neighborhood.append()
#     except:
#         KeyError

# results_df['hood_label'] = neighborhood


In [26]:
# results_df.dtypes
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)

In [27]:
# all_neighborhoods = get_all_neighborhoods()
# neighborhood = []
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)


    
# for row in results_df["longitude"]:
#     test_long = float(row)
    
#     print(test_long)
    

In [28]:

# neighborhood.append(find_neighborhood(test_long,test_lat,all_neighborhoods))
    
# print(neighborhood)
    

# results_df['hood_label'] = neighborhood

# for row in results_df
# #     test_long = (results_df["longitude"])
# #     test_lat = (results_df["latitude"])
#     neighborhood.append(find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods))
    
# print(neighborhood)

In [29]:
# all_neighborhoods = get_all_neighborhoods()
# neighborhood = []
# results_df['longitude'].astype(object).astype(float)
# results_df['latitude'].astype(object).astype(float)

# for row in results_df:
#      test_long = (results_df["longitude"])
#      test_lat = (results_df["latitude"])
#     find_neighborhood(results_df["longitude"],results_df["latitude"],all_neighborhoods)
    
# print(neighborhood)